<a href="https://colab.research.google.com/github/woo-jungnam/TH-TTNT/blob/main/TTNT_Tuan3_2001230540_NguyenThanhNam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import copy
import math
import random
import numpy
# Biểu diễn người chơi và ô trống
X = "X"
O = "O"
EMPTY = None

# Biến toàn cục sẽ được gán trong phần main (người dùng chọn X hoặc O)
user = None
ai = None

def initial_state():
    """
    Trả về trạng thái ban đầu của bàn cờ: ma trận 3x3 tất cả là EMPTY.
    """
    return [[EMPTY, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY]]

def player(board):
    """
    Trả về người chơi có lượt đi tiếp theo trên board.
    Cách xác định: đếm số ô đã được đánh; nếu số lượng ô đã đánh lẻ => hiện tại là lượt của ai (biến toàn cục 'ai'),
    ngược lại là lượt của user.
    Lưu ý: hàm này giả sử biến toàn cục 'user' và 'ai' đã được gán giá trị trước khi gọi.
    """
    count = 0
    for i in board:
        for j in i:
            # Nếu ô không rỗng (X hoặc O), tăng bộ đếm
            if j:
                count += 1
    # Nếu số ô đã đánh lẻ -> lượt tiếp theo là ai (vì user luôn đi trước nếu số ô là chẵn)
    if count % 2 != 0:
        return ai
    return user

def actions(board):
    """
    Trả về tập các hành động hợp lệ trên board hiện tại.
    Mỗi hành động là một cặp (i, j) chỉ vị trí hàng, cột có giá trị EMPTY.
    Dùng set để tránh trùng lặp và để dễ duyệt.
    """
    res = set()
    board_len = len(board)
    for i in range(board_len):
        for j in range(board_len):
            if board[i][j] == EMPTY:
                res.add((i, j))
    return res

def result(board, action):
    """
    Trả về board mới được tạo bằng cách áp dụng action lên board (không thay đổi board gốc).
    - Sao chép sâu board bằng copy.deepcopy
    - Đặt ký hiệu của người chơi hiện tại vào ô (i, j)
    """
    curr_player = player(board)
    result_board = copy.deepcopy(board)
    (i, j) = action
    result_board[i][j] = curr_player
    return result_board

def get_horizontal_winner(board):
    """
    Kiểm tra thắng hàng ngang:
    - Duyệt từng hàng, lấy giá trị ô đầu tiên của hàng làm ứng viên (winner_val).
    - So sánh tất cả ô trong hàng với winner_val; nếu có ô khác (hoặc None) => không thắng ở hàng đó.
    - Nếu cả hàng đều giống nhau (và khác None) => trả về ký hiệu thắng (X hoặc O).
    """
    winner_val = None
    board_len = len(board)
    for i in range(board_len):
        winner_val = board[i][0]
        for j in range(board_len):
            if board[i][j] != winner_val:
                winner_val = None
                break  # thoát vòng trong nếu gặp ô khác
        if winner_val:
            return winner_val
    return winner_val  # None nếu không có hàng thắng

def get_vertical_winner(board):
    """
    Kiểm tra thắng theo cột (tương tự get_horizontal_winner nhưng theo cột).
    """
    winner_val = None
    board_len = len(board)
    for i in range(board_len):
        winner_val = board[0][i]
        for j in range(board_len):
            if board[j][i] != winner_val:
                winner_val = None
                break
        if winner_val:
            return winner_val
    return winner_val

def get_diagonal_winner(board):
    """
    Kiểm tra thắng theo hai đường chéo:
    - Main diagonal (ô [0,0], [1,1], [2,2])
    - Anti-diagonal (ô [0,2], [1,1], [2,0])
    Nếu một trong hai chéo có 3 ký hiệu giống nhau (và khác None) => trả về ký hiệu đó.
    """
    winner_val = None
    board_len = len(board)

    # Kiểm tra chéo chính
    winner_val = board[0][0]
    for i in range(board_len):
        if board[i][i] != winner_val:
            winner_val = None
            break
    if winner_val:
        return winner_val

    # Kiểm tra chéo phụ (anti-diagonal)
    winner_val = board[0][board_len - 1]
    for i in range(board_len):
        j = board_len - 1 - i
        if board[i][j] != winner_val:
            winner_val = None
            break
    return winner_val  # None nếu không có đường chéo thắng

def winner(board):
    """
    Trả về người thắng (X hoặc O) nếu có, hoặc None nếu chưa có ai thắng.
    Kết hợp kiểm tra ngang, dọc, chéo.
    """
    winner_val = get_horizontal_winner(board) or get_vertical_winner(board) \
                 or get_diagonal_winner(board)
    return winner_val

def terminal(board):
    """
    Kiểm tra trạng thái kết thúc của trò chơi:
    - Nếu có người thắng => True
    - Nếu không còn ô trống nào => True (hòa)
    - Ngược lại => False (trò chơi tiếp tục)
    """
    if winner(board) is not None:
        return True
    for i in board:
        for j in i:
            if j == EMPTY:
                return False
    return True

def utility(board):
    """
    Hàm utility (giá trị trạng thái) dùng cho Minimax:
    - Trả về 1 nếu X thắng
    - Trả về -1 nếu O thắng
    - Trả về 0 nếu hòa hoặc chưa có người thắng
    Lưu ý: giả sử X luôn là người mà ta đánh giá là 'tốt' (max).
    """
    winner_val = winner(board)
    if winner_val == X:
        return 1
    elif winner_val == O:
        return -1
    return 0

def maxValue(state):
    """
    Hàm đệ quy cho Minimax: giá trị tối đa mà người chơi max (X) có thể đạt được từ state.
    - Nếu state là terminal -> trả về utility
    - Khởi tạo v = -inf, duyệt tất cả hành động, lấy max giữa v và minValue(kết quả hành động).
    """
    if terminal(state):
        return utility(state)
    v = -math.inf
    for action in actions(state):
        v = max(v, minValue(result(state, action)))
    return v

def minValue(state):
    """
    Hàm đệ quy cho Minimax: giá trị tối thiểu mà người chơi min (O) có thể ép max phải nhận.
    - Nếu state là terminal -> trả về utility
    - Khởi tạo v = +inf, duyệt hành động, lấy min giữa v và maxValue(kết quả hành động).
    """
    if terminal(state):
        return utility(state)
    v = math.inf
    for action in actions(state):
        v = min(v, maxValue(result(state, action)))
    return v

def minimax(board):
    """
    Trả về hành động tối ưu cho người chơi hiện tại trên board:
    - X được xem là maximizer (tìm giá trị lớn nhất)
    - O là minimizer (tìm giá trị nhỏ nhất)
    Lưu ý: hàm gọi maxValue/minValue để đánh giá giá trị của mỗi hành động.
    """
    current_player = player(board)
    if current_player == X:
        # Tìm hành động làm maxValue lớn nhất
        best_value = -math.inf
        best_action = None
        for action in actions(board):
            value = minValue(result(board, action))
            if value > best_value:
                best_value = value
                best_action = action
        return best_action
    else:  # current_player == O
        # Tìm hành động làm minValue nhỏ nhất
        best_value = math.inf
        best_action = None
        for action in actions(board):
            value = maxValue(result(board, action))
            if value < best_value:
                best_value = value
                best_action = action
        return best_action

if __name__ == "__main__":
    # Khởi tạo bàn cờ trống
    board = initial_state()
    ai_turn = False

    # Chọn người chơi (user nhập X hoặc O)
    print("Choose a player (X hoặc O). Gõ X hoặc O rồi Enter:")
    user = input().strip()
    if user == "X":
        ai = "O"
    else:
        ai = "X"

    # Vòng lặp chính của trò chơi
    while True:
        game_over = terminal(board)
        playr = player(board)

        if game_over:
            # Khi kết thúc trò chơi, in kết quả: thắng/thua/hòa
            winner_val = winner(board)
            if winner_val is None:
                print("Game Over: Tie.")
            else:
                print(f"Game Over: {winner_val} wins.")
            break
        else:
            # Nếu đến lượt AI (người dùng không phải player hiện tại)
            if user != playr and not game_over:
                if ai_turn:
                    # AI tính nước đi tối ưu bằng minimax
                    move = minimax(board)
                    board = result(board, move)
                    ai_turn = False
                    # In bàn cờ ra dưới dạng mảng numpy để dễ nhìn
                    print(numpy.array(board))
            # Nếu đến lượt người dùng
            elif user == playr and not game_over:
                ai_turn = True
                print("Enter your move as 'row col' (e.g., 0 1 for top-middle):")
                while True:
                    try:
                        user_input = input("Your move: ")
                        row_str, col_str = user_input.split()
                        i = int(row_str)
                        j = int(col_str)

                        # Kiểm tra hợp lệ của chỉ số
                        if not (0 <= i < 3 and 0 <= j < 3):
                            print("Invalid move. Row and column must be between 0 and 2.")
                            continue
                        # Nếu ô trống thì áp dụng nước đi
                        if board[i][j] == EMPTY:
                            board = result(board, (i, j))
                            print(numpy.array(board))
                            break
                        else:
                            print("Invalid move. Position already taken.")
                    except ValueError:
                        # Bắt lỗi khi người dùng nhập không đúng định dạng
                        print("Invalid input. Please enter two numbers separated by a space (e.g., 0 1).")
                    except IndexError:
                        print("Invalid input format. Please enter two numbers separated by a space (e.g., 0 1).")


Choose a player (X hoặc O). Gõ X hoặc O rồi Enter:
X
Enter your move as 'row col' (e.g., 0 1 for top-middle):
Your move: 0 1
[[None 'X' None]
 [None None None]
 [None None None]]
[[None 'X' None]
 [None None None]
 [None 'O' None]]
Enter your move as 'row col' (e.g., 0 1 for top-middle):
Your move: 0 0
[['X' 'X' None]
 [None None None]
 [None 'O' None]]
[['X' 'X' 'O']
 [None None None]
 [None 'O' None]]
Enter your move as 'row col' (e.g., 0 1 for top-middle):
Your move: 1 1
[['X' 'X' 'O']
 [None 'X' None]
 [None 'O' None]]
[['X' 'X' 'O']
 [None 'X' None]
 [None 'O' 'O']]
Enter your move as 'row col' (e.g., 0 1 for top-middle):
Your move: 1 2
[['X' 'X' 'O']
 [None 'X' 'X']
 [None 'O' 'O']]
[['X' 'X' 'O']
 [None 'X' 'X']
 ['O' 'O' 'O']]
Game Over: O wins.


In [11]:
import os, math

def GetWinner(board):
    """
    Trả về người thắng (chuỗi "X" hoặc "O") nếu có một dòng/ cột/chéo giống nhau 3 ô.
    Nếu không có thắng thì trả về None.

    Lưu ý: 'board' ở đây là danh sách phẳng 9 phần tử (chỉ số 0..8):
      0 1 2
      3 4 5
      6 7 8
    Các ô trống ban đầu được biểu diễn bằng số (1..9) trong chương trình gốc.
    """
    # ngang (horizontal): kiểm tra 3 hàng
    if board[0] == board[1] and board[1] == board[2] and board[0] is not None:
        return board[0]
    elif board[3] == board[4] and board[4] == board[5] and board[3] is not None:
        return board[3]
    elif board[6] == board[7] and board[7] == board[8] and board[6] is not None:
        return board[6]
    # dọc (vertical): kiểm tra 3 cột
    elif board[0] == board[3] and board[3] == board[6] and board[0] is not None:
        return board[0]
    elif board[1] == board[4] and board[4] == board[7] and board[1] is not None:
        return board[1]
    elif board[2] == board[5] and board[5] == board[8] and board[2] is not None:
        return board[2]
    # chéo (diagonal): 2 đường chéo chính
    elif board[0] == board[4] and board[4] == board[8] and board[0] is not None:
        return board[0]
    elif board[2] == board[4] and board[4] == board[6] and board[2] is not None:
        return board[2]
    # nếu không có người thắng, trả về None
    return None

def PrintBoard(board):
    """
    Xóa console (tùy OS) và in ra bảng 3x3 hiện tại.
    Gợi ý: nếu chạy trong IDE (ví dụ PyCharm) lệnh clear có thể không xóa được màn hình như terminal.
    """
    os.system('cls' if os.name=='nt' else 'clear')
    print(f'''
 {board[0]}|{board[1]}|{board[2]}
------
 {board[3]}|{board[4]}|{board[5]}
------
 {board[6]}|{board[7]}|{board[8]}
''')

def GetAvailableCells(board):
    """
    Trả về danh sách các ô chưa bị đánh (chứa số 1..9 trong cách biểu diễn hiện tại).
    - board: list 9 phần tử, ô trống là các số (1..9), ô bị chiếm bởi "X" hoặc "O".
    - Hàm trả về danh sách các giá trị cell (không phải chỉ số 0-based), vì mã gốc dùng cell để dễ hiển thị và gán lại.
    """
    available = list()
    for cell in board:
        # Nếu ô không phải "X" và không phải "O" thì coi là ô trống
        if cell != "X" and cell != "O":
            available.append(cell)
    return available

def minimax(position, depth, alpha, beta, isMaximizing):
    """
    Minimax đệ quy kèm Alpha-Beta pruning.
    Tham số:
      - position: board hiện tại (list 9 phần tử)
      - depth: độ sâu hiện tại (số nước đã đi kể từ trạng thái gốc của lần gọi minimax này)
      - alpha: best value (maximizer) đã tìm được (upper bound cho minimizer)
      - beta: best value (minimizer) đã tìm được (lower bound cho maximizer)
      - isMaximizing: boolean, True nếu lượt tiếp theo là lượt của Maximizer ("X"), False nếu là Minimizer ("O")

    Giá trị trả về:
      - numeric: giá trị đánh giá của vị trí hiện tại theo quy ước:
          +10 - depth  => nếu X thắng (càng nhanh càng tốt)
          -10 + depth  => nếu O thắng (càng muộn càng tốt cho X)
           0          => hoà
    Lưu ý về depth: sử dụng depth để ưu tiên chiến thắng nhanh hơn và trì hoãn thua lâu hơn.
    """
    # Kiểm tra người thắng hiện tại
    winner = GetWinner(position)
    if winner is not None:
        # Nếu X thắng => trả về 10 - depth (càng sâu thì trừ càng lớn => ưu tiên chiến thắng sớm)
        # Nếu O thắng => trả về -10 + depth (càng sâu thì cộng càng lớn => ưu tiên trì hoãn thua)
        return (10 - depth) if winner == "X" else (-10 + depth)

    # Nếu không còn ô trống => hoà
    if len(GetAvailableCells(position)) == 0:
        return 0

    # Nếu là lượt của người tối đa hoá (X)
    if isMaximizing:
        maxEval = -math.inf
        # Duyệt tất cả ô trống (giá trị cell là 1..9)
        for cell in GetAvailableCells(position):
            # Thử đánh X vào ô cell (cell-1 vì danh sách là 0-based)
            position[cell - 1] = "X"
            # Đệ quy: sau khi X đánh, lượt tiếp theo là minimizer (False)
            Eval = minimax(position, depth + 1, alpha, beta, False)
            # cập nhật giá trị tốt nhất cho maximizer
            maxEval = max(maxEval, Eval)
            # cập nhật alpha (best của maximizer)
            alpha = max(alpha, Eval)
            # revert (đặt lại ô về số ban đầu để thử nước khác)
            position[cell - 1] = cell
            # Alpha-Beta prune: nếu alpha >= beta thì các nhánh còn lại không cần xét
            if beta <= alpha:
                break  # prune
        return maxEval
    else:
        # Lượt của minimizer (O)
        minEval = +math.inf
        for cell in GetAvailableCells(position):
            position[cell - 1] = "O"
            # Đệ quy: sau khi O đánh, lượt tiếp theo là maximizer (True)
            Eval = minimax(position, depth + 1, alpha, beta, True)
            minEval = min(minEval, Eval)
            # cập nhật beta (best của minimizer)
            beta = min(beta, Eval)
            position[cell - 1] = cell
            if beta <= alpha:
                break  # prune
        return minEval

def FindBestMove(currentPosition, AI):
    """
    Tìm nước đi tốt nhất cho AI hiện tại.
    - currentPosition: board hiện tại (list 9 phần tử)
    - AI: ký hiệu AI ("X" hoặc "O")
    Trả về: bestMove (int) là giá trị cell (1..9) tốt nhất.

    Cách hoạt động:
      - Duyệt từng ô trống, giả sử AI đánh vào ô đó, gọi minimax để đánh giá
      - Với AI == "X" ta muốn maximize (giá trị lớn hơn tốt hơn)
      - Với AI == "O" ta muốn minimize (giá trị nhỏ hơn tốt hơn)
    """
    # Khởi tạo bestVal phụ thuộc AI là maximizer hay minimizer
    bestVal = -math.inf if AI == "X" else +math.inf
    bestMove = -1

    for cell in GetAvailableCells(currentPosition):
        # Thử đặt AI vào ô cell
        currentPosition[cell - 1] = AI
        # Gọi minimax:
        #   - nếu AI == "X" (maximizer) thì sau khi AI đặt xong, lượt tiếp theo là minimizer => isMaximizing=False
        #   - nếu AI == "O" (minimizer) thì sau khi AI đặt xong, lượt tiếp theo là maximizer => isMaximizing=True
        moveVal = minimax(currentPosition, 0, -math.inf, +math.inf,
                          False if AI == "X" else True)
        # revert ô
        currentPosition[cell - 1] = cell

        # cập nhật bestMove theo moveVal
        if (AI == "X" and moveVal > bestVal):
            bestMove = cell
            bestVal = moveVal
        elif (AI == "O" and moveVal < bestVal):
            bestMove = cell
            bestVal = moveVal

    return bestMove

def main():
    """
    Vòng lặp điều khiển trò chơi:
      - Người chơi nhập chọn X hoặc O
      - Board biểu diễn bằng danh sách 1..9 ban đầu
      - X luôn đi trước
      - Lần lượt AI và người chơi đi cho đến khi có người thắng hoặc đầy ô
    """
    player = input("Play as X or O? ").strip().upper()
    AI = "O" if player == "X" else "X"

    # currentGame lưu trạng thái: ban đầu là số 1..9 biểu diễn ô trống
    currentGame = [*range(1, 10)]
    # X bắt đầu trước
    currentTurn = "X"
    counter = 0

    while True:
        if currentTurn == AI:
            # Nước đi AI: tìm ô tốt nhất rồi gán
            # Ghi chú: nếu AI đi trước (AI == "X" và currentGame là trạng thái khởi tạo),
            # FindBestMove có thể luôn trả về góc/tâm tuỳ triển khai; trong code gốc tác giả ghi chú rằng AI sẽ chọn index 0.
            cell = FindBestMove(currentGame, AI)
            currentGame[cell - 1] = AI
            currentTurn = player
        elif currentTurn == player:
            # Lượt người: in board, đọc input, validate, rồi đặt
            PrintBoard(currentGame)
            while True:
                try:
                    humanInput = int(input("Enter Number (1-9): ").strip())
                    if 1 <= humanInput <= 9:
                        # Nếu ô chưa bị chiếm (không phải "X" hoặc "O")
                        if currentGame[humanInput - 1] != "X" and currentGame[humanInput - 1] != "O":
                            currentGame[humanInput - 1] = player
                            currentTurn = AI
                            break
                        else:
                            PrintBoard(currentGame)
                            print("Cell Not Available or already taken.")
                    else:
                        PrintBoard(currentGame)
                        print("Invalid number. Please enter a number between 1 and 9.")
                except ValueError:
                    # Bắt lỗi nếu người dùng không nhập số
                    PrintBoard(currentGame)
                    print("Invalid input. Please enter a number.")

        # Sau mỗi nước, kiểm tra thắng/hoà
        if GetWinner(currentGame) is not None:
            PrintBoard(currentGame)
            print(f"{GetWinner(currentGame)} WON!!!")
            break
        counter += 1
        # Nếu đã đi 9 nước mà không có winner -> hoà
        if GetWinner(currentGame) is None and counter == 9:
            PrintBoard(currentGame)
            print("Tie.")
            break

if __name__ == "__main__":
    main()


Play as X or O? X

 1|2|3
------
 4|5|6
------
 7|8|9

Enter Number (1-9): 2

 O|X|3
------
 4|5|6
------
 7|8|9

Enter Number (1-9): 3

 O|X|X
------
 O|5|6
------
 7|8|9

Enter Number (1-9): 5

 O|X|X
------
 O|X|6
------
 O|8|9

O WON!!!
